In [1]:
!pip install langgraph langchain-openai langchain-community langchain-tavily

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import Literal, TypedDict, List, Optional

# 체인 및 기타 함수들
from chain_setting import create_chain
from normalize_code_search import parse_extracted_text
from normalize_code_search import find_corporation_code, parse_extracted_text
from retreiver_setting import faiss_retriever_loading

accounting_retriever, business_retriever, business_retriever2, self_retriever = faiss_retriever_loading()

C:\Workspaces\JeongMinYoung\utils1\retreiver_setting.py:71: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)


In [3]:
from api_get import get_financial_state

In [4]:
from normalize_code_search import parse_extracted_text

In [5]:
from chain_setting import create_chain

simple_chain, classification_chain, extract_chain,hybrid_chain1, hybrid_chain2, hybrid_chain3,account_chain1, account_chain2, account_chain3,business_chain1, business_chain2, business_chain3, financial_chain1, financial_chain2, financial_chain3 = create_chain()

In [6]:
from typing import TypedDict, Literal, List, Optional

# 타입 정의
class ChatTurn(TypedDict):
    role: Literal["user", "assistant"]
    content: str

class ChatState(TypedDict, total=False):
    question: str
    company: Optional[str]
    year: Optional[List[str]]
    route: Optional[str]
    answer: Optional[str]
    chat_history: List[ChatTurn]

In [7]:
def handle_accounting1(state: ChatState) -> dict:
    question = state["question"]
    docs = accounting_retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    answer = account_chain1.invoke({"context": context, "question": question})
    updated_history = state.get("chat_history", []).copy()
    updated_history.append({"role": "user", "content": question})
    updated_history.append({"role": "assistant", "content": answer})

    return {
        **state,
        "answer": answer,
        "chat_history": updated_history  # ✅ 추가
    }

In [8]:
def extract_entities(state: ChatState) -> dict:
    question = state["question"]
    prev_company = state.get("company")
    prev_years = state.get("year")

    result = extract_chain.invoke({"question": question})
    # result는 문자열이므로 바로 사용
    parsed = parse_extracted_text(result)  # ✅ result["text"] → result

    company = parsed.get("company") or prev_company
    years = parsed.get("year_list") or prev_years or ["2024"]

    return {
        **state,
        "company": company,
        "year": years,
    }

In [9]:
def handle_financial1(state: ChatState) -> dict:
    question = state["question"]
    company = state["company"]
    years = state.get("year", ["2024"])

    corp_code = find_corporation_code(company)
    if corp_code.startswith("[ERROR]"):
        return {**state, "answer": corp_code}

    fin_blocks = []
    for y in years:
        rows = get_financial_state(corp_code, y, "11011", "CFS")
        if rows:
            fin_blocks.append(f"📅 {y}년 재무제표:\n" + "\n".join(rows))

    structured_financial = "\n\n".join(fin_blocks)
    answer = financial_chain1.invoke({
        "financial_data": structured_financial,
        "question": question,
        "resolved_corp_name": company,
    })

    updated_history = state.get("chat_history", []).copy()
    updated_history.append({"role": "user", "content": question})
    updated_history.append({"role": "assistant", "content": answer})

    return {
        **state,
        "answer": answer,
        "chat_history": updated_history  # ✅ 추가
    }

In [10]:
def handle_business1(state: ChatState) -> dict:
    question = state["question"]
    docs = self_retriever.get_relevant_documents(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    answer = business_chain1.invoke({"context": context, "question": question})
    updated_history = state.get("chat_history", []).copy()
    updated_history.append({"role": "user", "content": question})
    updated_history.append({"role": "assistant", "content": answer})

    return {
        **state,
        "answer": answer,
        "chat_history": updated_history  # ✅ 추가
    }

In [11]:
def handle_hybrid1(state: ChatState) -> dict:
    question = state["question"]
    company = state["company"]
    years = state.get("year", ["2024"])

    corp_code = find_corporation_code(company)
    if corp_code.startswith("[ERROR]"):
        return {**state, "answer": corp_code}

    def try_get_financial_strict(corp_code: str, year: str) -> str:
        rows = get_financial_state(corp_code, year, "11011", "CFS")
        if rows and "[API 오류]" not in rows[0]:
            return f"📅 {year}년 (CFS):\n" + "\n".join(rows)
        return f"📅 {year}년 재무제표: 유효한 데이터를 찾을 수 없습니다."

    fin = "\n\n".join([try_get_financial_strict(corp_code, y) for y in years])
    acct_docs = accounting_retriever.invoke(question)
    biz_docs = business_retriever.invoke(question)

    acct = "\n\n".join(doc.page_content for doc in acct_docs)
    biz = "\n\n".join(doc.page_content for doc in biz_docs)

    answer = hybrid_chain1.invoke({
        "question": question,
        "acct": acct,
        "biz": biz,
        "fin": fin
    })

    updated_history = state.get("chat_history", []).copy()
    updated_history.append({"role": "user", "content": question})
    updated_history.append({"role": "assistant", "content": answer})

    return {
        **state,
        "answer": answer,
        "chat_history": updated_history  # ✅ 추가
    }

In [12]:
def elief(state: ChatState) -> dict:
    question = state["question"]
    chat_history = state.get("chat_history", [])

    # chat_history를 포맷하여 함께 전달
    response = simple_chain.invoke({
        "question": question,
        "chat_history": chat_history
    })

    # chat_history에 assistant 응답도 추가
    chat_history.append({"role": "assistant", "content": response})

    return {
        **state,
        "answer": response,
        "chat_history": chat_history
    }

In [13]:
def route_from_extract(state: ChatState) -> str:
    mapping = {
        "accounting": "accounting",
        "finance": "financial",
        "business": "business",
        "hybrid": "hybrid",
    }
    return mapping.get(state.get("route", "").strip(), "simple")

In [14]:
def classify(state: ChatState) -> dict:
    question = state["question"]
    chat_history = state.get("chat_history", [])

    # 최근 대화 몇 개만 context로 붙임
    context = "\n".join([f"{turn['role']}: {turn['content']}" for turn in chat_history[-4:]])

    result = classification_chain.invoke({
        "question": question,
        "context": context
    }).strip()

    return {
    **state,
    "route": result
}

def route_from_classify(state):
    route = state.get("route", "").strip()
    # classification_chain이 실제로 뭘 반환하는지에 따라 매핑
    if route in ["accounting", "finance", "business", "hybrid"]:
        return route
    return "simple"

In [15]:
# ✅ LangGraph 정의
graph = StateGraph(ChatState)

# 노드 등록
graph.add_node("classify", classify)
graph.add_node("extract", extract_entities)
graph.add_node("accounting", handle_accounting1)
graph.add_node("financial", handle_financial1)
graph.add_node("business", handle_business1)
graph.add_node("hybrid", handle_hybrid1)
graph.add_node("simple", elief)

# 흐름 정의
graph.set_entry_point("classify")

graph.add_conditional_edges(
    "classify",
    route_from_classify,
    {
        "accounting": "extract",  # extract를 거쳐서 처리
        "finance": "extract",
        "business": "extract",
        "hybrid": "extract",
        "simple": "simple"  # simple은 바로 처리
    }
)
# extract 결과에 따라 분기
graph.add_conditional_edges(
    "extract",
    route_from_extract,
    {
        "accounting": "accounting",
        "financial": "financial",
        "business": "business",
        "hybrid": "hybrid"
    }
)

# 종료 노드 지정
for node in ["accounting", "financial", "business", "hybrid", "simple"]:
    graph.add_edge(node, END)

In [16]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
compiled_graph = graph.compile(checkpointer=memory)

# 동일한 thread_id로 계속 대화
config = {"configurable": {"thread_id": "conversation_1"}}

# 첫 번째 질문
result1 = compiled_graph.invoke({
    "question": "삼성전자의 2024년 재무제표를 알려줘"
}, config=config)

# 두 번째 질문 - 자동으로 이전 대화 기억
result2 = compiled_graph.invoke({
    "question": "그럼 2023년은 어땠어?"
}, config=config)

In [17]:
config = {"configurable": {"thread_id": "conversation_1"}}

In [21]:
input_state = {
    "question": "LG전자 2024년 재무제표",
}


In [22]:
result3 = compiled_graph.invoke(input_state,
                                config=config)

In [23]:
result3

{'question': 'LG전자 2024년 재무제표',
 'company': 'LG전자',
 'year': ['2024'],
 'route': 'finance',
 'answer': '안녕하세요! LG전자의 2024년 성적표를 쉽고 간단하게 알려드릴게요.\n\n**✅ 돈은 작년보다 잘 벌었나요?**\n네, 잘 벌었어요!\n- **매출** (회사의 1년 총 판매금액)은 **6.6% 늘어난 87,728억원**을 기록했고요,\n- **영업이익** (장사해서 실제로 남긴 돈)은 **6.4% 줄어든 3,420억원**이에요. 작년보다는 조금 덜 벌었지만 여전히 좋은 성과에요!\n\n**✅ 회사는 튼튼한가요?**\n네, 튼튼해요!\n- **자산** (회사가 가진 모든 재산)은 **8.9% 늘어난 65,629억원**이고요,\n- **부채** (갚아야 할 돈)는 **10% 늘어난 40,418억원**이에요. 자산이 더 많이 늘어서 여전히 안정적이에요.\n\n**한마디로,** LG전자는 2024년에 돈도 잘 벌고 회사도 더 튼튼해졌어요!\n\n더 궁금한 점 있으시면 편하게 물어보세요.',
 'chat_history': [{'role': 'user', 'content': '삼성전자의 2024년 재무제표를 알려줘'},
  {'role': 'assistant',
   'content': '안녕하세요! 삼성전자의 2024년 성적표를 쉽고 간단하게 알려드릴게요.\n\n**✅ 돈은 작년보다 잘 벌었나요?**\n네, 아주 잘 벌었어요!\n- **매출** (회사의 1년 총 판매금액)은 **16.2% 늘어난 3,008,709억원**을 기록했고요,\n- **영업이익** (장사해서 실제로 남긴 돈)은 무려 **398.3%나 늘어난 32,726억원**이에요. 작년보다 장사를 훨씬 잘했네요!\n\n**✅ 회사는 튼튼한가요?**\n네, 튼튼해요!\n- **자산** (회사가 가진 모든 재산)은 **12.9% 늘어난 5,145,319억원**이고요,\n- **부채** (갚아야 할 돈)는 **21.8% 늘어난

In [18]:
input_state = {
    "question": "삼성전자의 2024년 재무제표를 알려줘",
}

# 사용자별 실행
config = {"configurable": {"thread_id": "user_123"}}
result = compiled_graph.invoke(input_state, config=config)

In [19]:
result

{'question': '삼성전자의 2024년 재무제표를 알려줘',
 'company': '삼성전자',
 'year': ['2024'],
 'route': 'finance',
 'answer': '안녕하세요! 삼성전자의 2024년 성적표를 쉽고 간단하게 알려드릴게요.\n\n**✅ 돈은 작년보다 잘 벌었나요?**\n네, 아주 잘 벌었어요!\n- **매출** (회사의 1년 총 판매금액)은 **16.2% 늘어난 3,008,709억원**을 기록했고요,\n- **영업이익** (장사해서 실제로 남긴 돈)은 무려 **398.3%나 늘어난 32,726억원**이에요. 작년보다 장사를 훨씬 잘했네요!\n\n**✅ 회사는 튼튼한가요?**\n네, 튼튼해요!\n- **자산** (회사가 가진 모든 재산)은 **12.9% 늘어난 5,145,319억원**이고요,\n- **부채** (갚아야 할 돈)는 **21.8% 늘어난 1,123,399억원**이에요. 자산에 비하면 아주 적은 수준이에요.\n\n**한마디로,** 삼성전자는 2024년에 돈도 훨씬 잘 벌고 회사도 더 튼튼해졌어요!\n\n더 궁금한 점 있으시면 편하게 물어보세요.',
 'chat_history': [{'role': 'user', 'content': '삼성전자의 2024년 재무제표를 알려줘'},
  {'role': 'assistant',
   'content': '안녕하세요! 삼성전자의 2024년 성적표를 쉽고 간단하게 알려드릴게요.\n\n**✅ 돈은 작년보다 잘 벌었나요?**\n네, 아주 잘 벌었어요!\n- **매출** (회사의 1년 총 판매금액)은 **16.2% 늘어난 3,008,709억원**을 기록했고요,\n- **영업이익** (장사해서 실제로 남긴 돈)은 무려 **398.3%나 늘어난 32,726억원**이에요. 작년보다 장사를 훨씬 잘했네요!\n\n**✅ 회사는 튼튼한가요?**\n네, 튼튼해요!\n- **자산** (회사가 가진 모든 재산)은 **12.9% 늘어난 5,145,319억원**이고요,\n- **부채** (갚아야 

In [97]:
result = compiled_graph.invoke(input_state, config=config)

In [98]:
result

{'question': '삼성전자의 2024년 재무제표를 알려줘',
 'company': '삼성전자',
 'year': ['2024'],
 'route': 'finance',
 'answer': '안녕하세요! 삼성전자의 2024년 성적표를 쉽고 간단하게 알려드릴게요.\n\n**✅ 돈은 작년보다 잘 벌었나요?**\n네, 아주 잘 벌었어요!\n- **매출** (회사의 1년 총 판매금액)은 **16.2% 늘어난 3,008,709억원**을 기록했고요,\n- **영업이익** (장사해서 실제로 남긴 돈)은 무려 **398.3%나 늘어난 32,726억원**이에요. 작년보다 장사를 훨씬 잘했네요!\n\n**✅ 회사는 튼튼한가요?**\n네, 튼튼해요!\n- **자산** (회사가 가진 모든 재산)은 **12.9% 늘어난 5,145,319억원**이고요,\n- **부채** (갚아야 할 돈)는 **21.8% 늘어난 1,123,399억원**이에요. 자산에 비하면 아주 적은 수준이에요.\n\n**한마디로,** 삼성전자는 2024년에 돈도 훨씬 잘 벌고 회사도 더 튼튼해졌어요!\n\n더 궁금한 점 있으시면 편하게 물어보세요.',
 'chat_history': [{'role': 'user', 'content': '삼성전자의 2024년 재무제표를 알려줘'},
  {'role': 'assistant',
   'content': '안녕하세요! 삼성전자의 2024년 성적표를 쉽고 간단하게 알려드릴게요.\n\n**✅ 돈은 작년보다 잘 벌었나요?**\n네, 아주 잘 벌었어요!\n- **매출** (회사의 1년 총 판매금액)은 **16.2% 늘어난 3,008,709억원**을 기록했고요,\n- **영업이익** (장사해서 실제로 남긴 돈)은 무려 **398.3%나 늘어난 32,726억원**이에요. 작년보다 장사를 훨씬 잘했네요!\n\n**✅ 회사는 튼튼한가요?**\n네, 튼튼해요!\n- **자산** (회사가 가진 모든 재산)은 **12.9% 늘어난 5,145,319억원**이고요,\n- **부채** (갚아야 